In [47]:
import csv
import json
import shapely.geometry

Read in several-Mb-large sidewalks geojson

In [49]:
sidewalk_fc = {'type': 'FeatureCollection', 'features': []}
with open('../data/sidewalkgeometries.tsv') as f:
    for row in csv.reader(f, delimiter='\t'):
        geom = json.loads(row[1])
        properties = {'id': int(row[0]),
                      'stroke-color': '#ddd',
                      'stroke-opacity': 1,
                      'stroke-width': 8}
        feature = {'type': 'Feature', 'geometry': geom, 'properties': properties}
        sidewalk_fc['features'].append(feature)

Get just the sidewalks in the West Woodland neighborhood

In [50]:
# West, South, East, North coordinates
#greenlake_wsen = [-122.334073,47.674548,-122.319094,47.68183]
cd_box = shapely.geometry.Polygon([[-122.3046970367,47.5979667018],
                                   [-122.3046970367,47.6006000885],
                                   [-122.2997403145,47.6006000885],
                                   [-122.2997403145,47.5979667018],
                                   [-122.3046970367,47.5979667018]])

cd_fc = {'type': 'FeatureCollection', 'features': []}
for feature in sidewalk_fc['features']:
    if cd_box.intersects(shapely.geometry.shape(feature['geometry'])):

        cd_fc['features'].append(feature)

Write to file

In [51]:
with open('../data/sidewalks_data_cd.geojson', 'w') as g:
    json.dump(cd_fc, g)

Combine with places where heuristic model disagrees with crowdsourcing

In [52]:
with open('../cse546/learndata-errorsHeuristic_onCrowd.geojson') as h:
    error_dat = json.load(h)

for feature in error_dat['features']:
    if cd_box.intersects(shapely.geometry.shape(feature['geometry'])):
        feature['properties']['stroke-opacity'] = 1
        feature['stroke-width'] = 8
        if feature['properties']['type'] == 'false negative':
            feature['stroke-color'] = '#0af'
        cd_fc['features'].append(feature)

In [53]:
with open('../figures/figure-heuristic-errors.geojson', 'w') as i:
    json.dump(cd_fc, i)